In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import missingno as msno
import seaborn as sns
from sklearn import (ensemble, preprocessing, tree,)
from sklearn.metrics import (auc, confusion_matrix, roc_auc_score, roc_curve,)
from sklearn.model_selection import (train_test_split, StratifiedKFold,)
from yellowbrick.classifier import (ConfusionMatrix, ROCAUC,)
from yellowbrick.model_selection import (LearningCurve,)

import pandas_profiling

In [ ]:
def normalize_data(data):
    
    data['Year'] = pd.DatetimeIndex(data['Date']).year
    data['Month'] = pd.DatetimeIndex(data['Date']).month
    data['Day'] = pd.DatetimeIndex(data['Date']).day
    
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    mappings2 = {'0':0, 'Jan,Apr,Jul,Oct':1, 'Feb,May,Aug,Nov':2, 'Mar,Jun,Sept,Dec':3}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    data.PromoInterval.replace(mappings2, inplace=True)
    data['StoreType'] = data['StoreType'].astype(int)
    data['Assortment'] = data['Assortment'].astype(int)
    data['StateHoliday'] = data['StateHoliday'].astype(int)
    data['PromoInterval'] = data['PromoInterval'].astype(int)

In [ ]:
def normalize_store(data):
    
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    mappings2 = {'0':0, 'Jan,Apr,Jul,Oct':1, 'Feb,May,Aug,Nov':2, 'Mar,Jun,Sept,Dec':3}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.PromoInterval.replace(mappings2, inplace=True)
    data['StoreType'] = data['StoreType'].astype(int)
    data['Assortment'] = data['Assortment'].astype(int)
    data['PromoInterval'] = data['PromoInterval'].astype(int)

In [ ]:
train = pd.read_csv("/kaggle/input/rossmann-store-sales/train.csv", low_memory=False)
test = pd.read_csv("/kaggle/input/rossmann-store-sales/test.csv", low_memory=False)
store = pd.read_csv("/kaggle/input/rossmann-store-sales/store.csv", low_memory=False)

# données qui ne seront pas modifiées

trainy = pd.read_csv("/kaggle/input/rossmann-store-sales/train.csv", low_memory=False)
testy = pd.read_csv("/kaggle/input/rossmann-store-sales/test.csv", low_memory=False)
storey = pd.read_csv("/kaggle/input/rossmann-store-sales/store.csv", low_memory=False)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
test.isnull().sum()

In [ ]:
train.isnull().sum()

In [ ]:
train.hist(bins = 30, figsize=(20,20), color = 'r')

In [ ]:
test.hist(bins = 30, figsize=(20,20), color = 'g')

In [ ]:
store.hist(bins = 30, figsize=(20,20), color = 'b')

In [ ]:
correlations = train.corr()
f, ax = plt.subplots(figsize=(20,20))
sns.heatmap(correlations, annot = True)

In [ ]:
correlations = store.corr()
f, ax = plt.subplots(figsize=(20,20))
sns.heatmap(correlations, annot = True)

In [ ]:
correlations = test.corr()
f, ax = plt.subplots(figsize=(20,20))
sns.heatmap(correlations, annot = True)

In [ ]:
sns.heatmap(train.isnull())

In [ ]:
sns.heatmap(test.isnull())

In [ ]:
sns.heatmap(store.isnull())

In [ ]:
test.fillna(1,inplace=True)

store.CompetitionDistance = store.CompetitionDistance.fillna(store.CompetitionDistance.median())

store.fillna(0,inplace=True)

In [ ]:
train = pd.merge(train, store, on='Store')

test = pd.merge(test, store, on='Store')

test.reset_index(drop=True, inplace=True)

test.set_index('Id', inplace=True)

test.reset_index()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
test.isnull().sum()

In [ ]:
train.hist(bins = 30, figsize=(20,20), color = 'r')

In [ ]:
test.hist(bins = 30, figsize=(20,20), color = 'g')

In [ ]:
correlations = train.corr()
f, ax = plt.subplots(figsize=(20,20))
sns.heatmap(correlations, annot = True)

In [ ]:
correlations = test.corr()
f, ax = plt.subplots(figsize=(20,20))
sns.heatmap(correlations, annot = True)

In [ ]:
sns.heatmap(train.isnull())

In [ ]:
sns.heatmap(test.isnull())

In [ ]:
normalize_data(train)

normalize_data(test)

train = train.loc[train.Open == 1]

train.loc[train.Sales > 0].reset_index(drop=True) 

train = train.loc[train.Sales > 0]

train.reset_index(drop=True, inplace=True)

In [ ]:
axis = train.groupby('Month')[['Sales']].mean().plot(figsize=(10,5), marker = 'o')
axis.set_title('Average SALES per MONTH in Train')

In [ ]:
axis = train.groupby('Month')[['Customers']].mean().plot(figsize=(10,5), marker = 'o')
axis.set_title('Average CUSTOMERS per MONTH in Train')

In [ ]:
axis = train.groupby('Day')[['Sales']].mean().plot(figsize=(10,5), marker = 'o')
axis.set_title('Average SALES per DAY in Train')

In [ ]:
axis = train.groupby('Day')[['Customers']].mean().plot(figsize=(10,5), marker = 'o')
axis.set_title('Average CUSTOMERS per DAY in Train')

In [ ]:
sns.barplot(x = 'Promo', y = 'Sales', data = train)

In [ ]:
sns.barplot(x = 'Promo', y = 'Customers', data = train)

In [ ]:
train = train.drop(columns="Date")

y = train.Sales

train = train.drop(columns = 'Sales')

train = train.drop(columns = 'Customers')

# test

test = test.drop(columns="Date")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.hist(bins = 30, figsize=(20,20), color = 'r')

In [ ]:
test.hist(bins = 30, figsize=(20,20), color = 'g')

In [ ]:
normalize_store(store)

In [ ]:
store.hist(bins = 30, figsize=(20,20), color = 'b')

In [ ]:
train.columns

In [ ]:
cols = ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'Year', 'Month',
       'Day']

In [ ]:
test.columns

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
store_cols_all = ['Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
store_cols = ['Store', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear']

In [ ]:
train['CompetitionOpenSinceMonth'] = train['CompetitionOpenSinceMonth'].max() - train['CompetitionOpenSinceMonth']

train['CompetitionOpenSinceYear'] = train['CompetitionOpenSinceYear'].max() - train['CompetitionOpenSinceYear']

In [ ]:
test['CompetitionOpenSinceMonth'] = test['CompetitionOpenSinceMonth'].max() - test['CompetitionOpenSinceMonth']

test['CompetitionOpenSinceYear'] = test['CompetitionOpenSinceYear'].max() - test['CompetitionOpenSinceYear']

In [ ]:
store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].max() - store['CompetitionOpenSinceMonth']

store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].max() - store['CompetitionOpenSinceYear']

In [ ]:
store.head()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
X = train

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

# Definition de la formule
$${\rm RMSPE} = \frac{1}{n}\sqrt{\sum\limits_{i = 1}^n {{{\left( {\frac{{{y_i} - {{\hat y}_i}}}{{{y_i}}}} \right)}^2}}}$$

In [ ]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y-yhat)**2))
    return rmspe

def rmspe_xg(yhat, y):
    y = y.get_label()
    y = np.expm1(y)
    yhat = np.expm1(yhat)
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y-yhat)**2))
    return "rmspe", rmspe

def neg_rmspe(yhat, y):
    y = np.expm1(y)
    yhat = np.expm1(yhat)
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y-yhat)**2))
    return -rmspe

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

# rf=RandomForestClassifier(n_estimators=100, random_state = 42)
# rf.fit(X_train, y_train)
# rf.score(X_test, y_test)

In [ ]:
rf1=RandomForestClassifier()
params = {
    "max_features": [0.4, "auto"],
    "n_estimators": [15, 200],
    "min_samples_leaf": [1, 0.1],
    "random_state":[42],
}
cv=GridSearchCV(rf1, params, n_jobs=-1).fit(X_train, y_train)

cv.best_params_

In [ ]:
rf2=RandomForestClassifier(
**{
    "max_features": "auto",
    "n_estimators": 200,
    "min_samples_leaf": 0.1,
    "random_state":42,
}
)

In [ ]:


# modele de reference simple comme premiere reference

from sklearn.dummy import DummyClassifier

bm= DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_test, y_test) # accuracy

In [ ]:
# regression linéaire

from sklearn.linear_model import LinearRegression
 
linear = LinearRegression()
linear.fit(X_train, y_train)
linear.score(X_test, y_test)

In [ ]:
# RANDOM FOREST
# paramètres par defaut utilisé et n_jobs = -1 pour choix automatic de jobs

# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(n_jobs=-1, random_state=42)
# rf.fit(X_train, y_train)
# rf.score(X_test, y_test)

In [ ]:
# rf.predict(test)

In [ ]:
# test['Sales'] = rf.predict(test)

In [ ]:
# test.columns.values

In [ ]:
# submission = test['Sales']
# submission = submission.reset_index()
# submission.head()

In [ ]:
submission.to_csv("./submission_open.csv", sep=',', index=False)